In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2025 United Kingdom Research and Innovation
#  Copyright 2025 The University of Manchester
#  Copyright 2025 Danmarks Tekniske Universitet
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Luke Lozenski (DTU)
#                   Nalin Gupta   (UKRI-STFC)
#                   Rasmia Kulan  (UKRI-STFC)

# NikonDataReader Demo

Note: this demo is a work in progress and was created for the purpose of demonstrating a contibution for the reader hackathon!

Here demo how to read in Nikon Data, perform FDK reconstruciton, and perform model based image reconstruction utilizing a primal dual hybrid gradient.

The  NikonDataReader allows:

- Reading of geometry
- Reading of projections 
- Normalization based on the white level
- Reading in projections of interest
- Flipping the data 
- Reading of flat fields

## Data format: .xtekct file


## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from: https://zenodo.org/records/14996227
We acknowledge Diamond Light Source for time on Beamline I13-2 under Proposal mt9396-1

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = '/work3/ljolo/Reader_Hackathon/Nikon_1000_data/RUP_060225_XX_WMG_CCPIDATSETS_EZJB_2_1K.xtekct'

## Loading Geometry

In [ ]:
from cil.utilities.display import show2D, show_geometry
from cil.io import NikonDataReader

In [ ]:
reader = NikonDataReader(file_path)

We can call the `.get_geometry` command to load the geometry of the imaging system without loading the raw data files. 

In [ ]:
acq_geom = reader.get_geometry()
show_geometry(acq_geom)

In [ ]:
print(acq_geom)

## Loading Projections, Cropped Subsets of Interest, and Data Manipulation

To load and display a specific projection of interest we can use the show2D command with the specified projection of interest specified

In [ ]:
data = reader.read()
show2D(data, origin='upper-left')

By default the NikonDataReader argument `normalise` is `True`, which means all projections are loaded and normalised by the detector white level, which is stored in the .xtekct file as WhiteLevel. If you want to load the data without normalisation, specify `normalise=False`

In [ ]:
reader = NikonDataReader(file_name=file_path, normalise=False)
data = reader.read()
show2D(data, origin='upper-left')


Use the `roi` argument to load a subset of the data. `roi` should be passed as a dictionary e.g. `{'axis_labels_1': (start, end, step),'axis_labels_2': (start, end, step)}` with axis labels that describe the data dimension labels

To load a cropped subset of the data, change the start and end values. `'axis_label': -1` is a shortcut to load all elements along the axis.



In [ ]:
roi = {'horizontal':(120, 870, 1), 'vertical':-1}
reader = NikonDataReader(file_name=file_path, roi=roi)
data = reader.read()
show2D(data)


To load a binned subset of the data, change the step value. Here we use different binning for the horizontal and vertical dimensions which results in a different aspect ratio

In [ ]:
roi = {'horizontal':(None, None, 4), 'vertical':(None, None, 2)}
reader = NikonDataReader(file_name=file_path, roi=roi)
data = reader.read()
show2D(data)

We can also use the argument `fliplr=True` to flip all projections in the vertical axis. If we enable this option we see that the projection is flipped in the left-right direction

In [ ]:
reader = NikonDataReader(file_name=file_path, fliplr=True)
data = reader.read()
show2D(data)

# Pre-processing and Reconstruction

For details of the pre-processing steps performed, see our [CIL-Demos example which uses the same dataset](https://github.com/TomographicImaging/CIL-Demos/blob/main/demos/1_Introduction/03_preprocessing.ipynb)! However, in the CIL-demo the data had been converted into the CIL Nexus format first.

Here we load the data, perform conversion from transimission to absorption, and define the image and acquisiton geometries from the data loader.

In [ ]:
#from cil.recon import FDK
from cil.plugins.tigre import FBP
from cil.processors import TransmissionAbsorptionConverter
from cil.utilities.display import show2D
from cil.io import NikonDataReader
file_path = '/work3/ljolo/Reader_Hackathon/Nikon_1000_data/RUP_060225_XX_WMG_CCPIDATSETS_EZJB_2_1K.xtekct'
reader = NikonDataReader(file_name=file_path)
data = reader.read()
data_exp = TransmissionAbsorptionConverter()(data)
data_exp.reorder('tigre')
acq_geom = reader.get_geometry()
img_geom = acq_geom.get_ImageGeometry()

Then we can perform the FDK reconstruction

In [ ]:
fdk = FBP(img_geom, acq_geom)
fdk.set_input(data_exp)
fdk_recon = fdk.get_output()

In [ ]:
show2D(fdk_recon,slice_list=('horizontal_y', 480), title='Reconstruction FDK',  origin='upper-left')

We can also perform a model based image reconstruction with Total Variaiton regularizaiton and a nonnegativity constraint

In [ ]:
from cil.optimisation.functions import L2NormSquared, BlockFunction, MixedL21Norm, IndicatorBox
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.algorithms import PDHG
from cil.plugins.tigre.ProjectionOperator import ProjectionOperator
from cil.processors import TransmissionAbsorptionConverter
from cil.utilities.display import show2D
from cil.io import NikonDataReader

In [ ]:
file_path = '/work3/ljolo/Reader_Hackathon/Nikon_1000_data/RUP_060225_XX_WMG_CCPIDATSETS_EZJB_2_1K.xtekct'
roi = {'horizontal':(None, None, 4), 'vertical':(None, None, 4)}
reader = NikonDataReader(file_name=file_path, roi = roi)
data = reader.read()
data_exp = TransmissionAbsorptionConverter()(data)
data_exp.reorder('tigre')
acq_geom = reader.get_geometry()
img_geom = acq_geom.get_ImageGeometry()
A = ProjectionOperator(img_geom, acq_geom, device='gpu')
Grad = GradientOperator(img_geom)
K = BlockOperator(A, Grad)
alpha = 1.0
f1 = 0.5 * L2NormSquared(b=data_exp)
f2 = alpha * MixedL21Norm()
f = BlockFunction(f1, f2)
g = IndicatorBox(lower=0)


In [ ]:
normK = K.norm()
sigma = 1./normK
tau = 1./normK


In [ ]:
pdhg = PDHG(f=f, g=g, operator=K, sigma=sigma, tau=tau, update_objective_interval=5)
pdhg.run(40, verbose=2)

In [ ]:
show2D(pdhg.solution,slice_list=('horizontal_y', 120), title='TV regularisation',  origin='upper-left')